In [2]:
%%bash

#pip install shapely
#pip install pyproj
#pip install pyogrio
pip install 'geopandas[all]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.2 MB/s eta 0:00:00


In [3]:
%%bash
head -100 /content/Mo_pop_Sim.csv

UR,long,lat
R,-92.5062761212864,40.2792029061618
R,-92.5051318550842,40.276889612375
R,-92.5058899870041,40.2782423375098
R,-92.5198418278749,40.3277000831164
R,-92.524877366008,40.3295107045137
R,-92.5291823662756,40.3250413897606
R,-92.5324094395201,40.3249121413652
R,-92.526117330564,40.3264884161639
R,-92.5238984086017,40.3283689573053
R,-92.5249032416176,40.3271086309602
R,-92.5316566215904,40.3242213951089
R,-92.5230777749596,40.3245033699447
R,-92.5242509080343,40.3302713952819
R,-92.5237486887981,40.3242483650306
R,-92.530986265588,40.3245361361651
R,-92.5252466463363,40.3231027736977
R,-92.5219058607571,40.326671995696
R,-92.5826490381395,40.2515549341225
R,-92.5108897641274,40.2638229644672
R,-92.489846693235,40.2614590871247
R,-92.4959112103302,40.2680114210773
R,-92.5009400891376,40.2636468925299
R,-92.5039040265058,40.2688985252472
R,-92.4918817039017,40.2597362963358
R,-92.6257326039896,40.3292801694796
R,-92.6282872182202,40.3251945459803
R,-92.6234825313172,40.320922619

In [4]:
pip install geopandas

In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [6]:
# Read the shapefile
shapefile_path = "/content/MO_2018_Federally_Qualified_Health_Center_Locations.shp"  # Replace with the actual path
gdf = gpd.read_file(shapefile_path)

# Now 'gdf' is a GeoDataFrame containing the data from the shapefile
# You can inspect it using:
#print(gdf.head())

print(gdf.crs) # print the coordinate reference system
crs = gdf.crs
gdf = gdf.set_index("OBJECTID")


print(gdf.columns)
fqhc_df = gdf['geometry']
fqhc_df

EPSG:4326
Index(['Group_Name', 'Facility', 'Address', 'City', 'County', 'State', 'Zip',
       'Phone', 'Latitude', 'Longitude', 'Loc_Code', 'geometry'],
      dtype='object')


,geometry
OBJECTID,
1,POINT (-90.55472 38.43597)
2,POINT (-91.13403 37.71462)
3,POINT (-92.60144 38.16025)
4,POINT (-90.45724 36.77261)
5,POINT (-94.49886 38.9629)
...,...
193,POINT (-91.77391 37.94594)
194,POINT (-93.29166 37.2243)
195,POINT (-91.56258 36.56534)


In [7]:
# In Missouri it is roughly 56 miles per degree of longitude and 69 miles per degree of latitude. We'll use an approximation of 63 miles per unit for our circle.
DEGREES_PER_MILE = 1/ 63
circles_geom = gdf.buffer( 30 * DEGREES_PER_MILE)
fqhc_circles_df = gpd.GeoDataFrame(gdf, geometry=circles_geom, crs=crs)
fqhc_circles_df

<ipython-input-7-6f6b0be0e886>:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  circles_geom = gdf.buffer( 30 * DEGREES_PER_MILE)


,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code,geometry
OBJECTID,,,,,,,,,,,,
1,COMTREA,COMTREA Byrnes Mill Health Center,100 Osage Executive Circle,House Springs,Jefferson,MO,63051,6367893372,38.435946,-90.554678,MAP,"POLYGON ((-90.07853 38.43597, -90.08082 38.389..."
2,Missouri Highlands Health Care,Viburnum Medical Clinic,18 Viburnum Center Road,Viburnum,Iron,MO,65566,5732445406,37.714620,-91.133983,MAP,"POLYGON ((-90.65784 37.71462, -90.66013 37.667..."
3,Central Ozarks Medical Center,Central Ozarks Medical Center At The Lake,3870 Columbia Avenue,Osage Beach,Miller,MO,65065,5733027490,38.160258,-92.601463,MAP,"POLYGON ((-92.12525 38.16025, -92.12754 38.113..."
4,Missouri Highlands Health Care,Missouri Highland Medical Clinic - Poplar Bluf...,"225 Physicians Park Drive, Suite 303",Poplar Bluff,Butler,MO,63901,5737856536,36.772568,-90.457206,MAP,"POLYGON ((-89.98105 36.77261, -89.98334 36.725..."
5,Swope Health Services,Swope Health Hickman Mills,8800 Blue Ridge Boulevard,Kansas City,Jackson,MO,64138,8163213200,38.962882,-94.498847,MAP,"POLYGON ((-94.02267 38.9629, -94.02496 38.9162..."
...,...,...,...,...,...,...,...,...,...,...,...,...
193,Your Community Health Center,Your Community Health Center - Health Department,"200 North Main, Suite G51",Rolla,Phelps,MO,65401,5734586950,37.945925,-91.773948,MAP,"POLYGON ((-91.29772 37.94594, -91.30001 37.899..."
194,Jordan Valley Community Health Center,Jordan Valley Community Health Center - Medica...,"1443 N. Roberson, Suite 505",Springfield,Greene,MO,65802,4178511554,37.224321,-93.291591,MAP,"POLYGON ((-92.81547 37.2243, -92.81776 37.1776..."
195,Southern Missouri Community Health Center,Oregon County Community Health Center,"US Highway 63 North (RR 3, Box 3703)",Thayer,Oregon,MO,65791,4172642990,36.565336,-91.562618,MAP,"POLYGON ((-91.08639 36.56534, -91.08869 36.518..."


In [8]:
def create_point_column(df, lon_col, lat_col, crs="EPSG:4326"):
    """
    Creates a geometry column of Point objects from longitude and latitude columns in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing longitude and latitude columns.
        lon_col (str): The name of the longitude column.
        lat_col (str): The name of the latitude column.
        crs (str, optional): Coordinate Reference System. Defaults to "EPSG:4326".

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame with the added geometry column.
    """

    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

In [18]:
# Read the shapefile
pop_file_path = "/content/Mo_pop_Sim.csv"
big_pop_df = pd.read_csv(pop_file_path)
pop_df = big_pop_df.sample(frac=0.01)

pop_gdf = create_point_column(pop_df, 'long', 'lat', crs)
pop_gdf

print(pop_gdf)

        UR       long        lat                    geometry
1624348  U -93.311184  37.194882  POINT (-93.31118 37.19488)
3591686  R -94.845337  40.315306  POINT (-94.84534 40.31531)
5129042  U -90.337760  38.826442  POINT (-90.33776 38.82644)
379641   U -94.826897  39.778208   POINT (-94.8269 39.77821)
647122   U -89.567906  37.313923  POINT (-89.56791 37.31392)
...     ..        ...        ...                         ...
4343678  U -90.749226  38.756066  POINT (-90.74923 38.75607)
3003094  U -90.411932  38.213449  POINT (-90.41193 38.21345)
6231858  U -90.281368  38.655472  POINT (-90.28137 38.65547)
1694780  U -93.345073  37.116285  POINT (-93.34507 37.11629)
4532906  U -90.408884  37.778772  POINT (-90.40888 37.77877)

[63325 rows x 4 columns]


In [10]:
len(pop_gdf)

47463

In [46]:
pop_in_circles = gpd.sjoin(pop_gdf, fqhc_circles_df, predicate='within').groupby('OBJECTID').count()
pop_in_circles

,UR,long,lat,geometry,Group_Name,Facility,Address,City,County,State,Zip,Phone,Latitude,Longitude,Loc_Code
OBJECTID,,,,,,,,,,,,,,,
1,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
2,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
3,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
4,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
5,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
194,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325
195,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325,63325


In [51]:
# Buffer for 30 miles around FQHC locations
MILES_TO_FEET = 5280
circles_geom = gdf.buffer(30 * MILES_TO_FEET)
fqhc_circles_df = gpd.GeoDataFrame(gdf, geometry=circles_geom)

# Assuming pop_gdf is already in the correct coordinate reference system (CRS)
# If it's not, you might need to adjust this, but it's not necessary for the task.
pop_gdf_projected = pop_gdf

pop_in_circles = gpd.sjoin(pop_gdf, fqhc_circles_df, predicate='within').groupby('OBJECTID').count()
# Check for duplicates in pop_gdf
pop_gdf = pop_gdf.drop_duplicates(subset=['geometry'])

<ipython-input-51-fc3f9e5ec978>:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  circles_geom = gdf.buffer(30 * MILES_TO_FEET)


In [53]:
# Fitness Function (Density Approach)
def calculate_fitness(fqhc_index, population_counts):
    return population_counts.get(fqhc_index, 0)

# Genetic Algorithm
def genetic_algorithm(population_size=100, generations=100, num_selected=8, mutation_rate=0.01):
    num_fqhcs = len(gdf)

    def create_individual():
        return np.random.choice(num_fqhcs, num_selected, replace=False)

    def calculate_population_fitness(population, population_counts):
        fitness_scores = []
        for individual in population:
            total_fitness = sum(calculate_fitness(fqhc_index + 1, population_counts) for fqhc_index in individual)
            fitness_scores.append(total_fitness)
        return np.array(fitness_scores)

    def selection(population, fitness_scores):
        probabilities = fitness_scores / np.sum(fitness_scores)
        selected_indices = np.random.choice(len(population), size=population_size, p=probabilities)
        return population[selected_indices]

    def crossover(parent1, parent2):
        crossover_point = np.random.randint(1, num_selected)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2

    def mutate(individual):
        for i in range(num_selected):
            if np.random.rand() < mutation_rate:
                new_fqhc = np.random.randint(num_fqhcs)
                while new_fqhc in individual:
                    new_fqhc = np.random.randint(num_fqhcs)
                individual[i] = new_fqhc
        return individual

    population = np.array([create_individual() for _ in range(population_size)])

    for generation in range(generations):
        fitness_scores = calculate_population_fitness(population, population_counts)
        selected_population = selection(population, fitness_scores)

        new_population = []
        for i in range(0, population_size, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([mutate(child1), mutate(child2)])

        population = np.array(new_population)

    best_individual_index = np.argmax(calculate_population_fitness(population, population_counts))
    best_individual = population[best_individual_index]
    best_fitness = calculate_population_fitness(population, population_counts)[best_individual_index]

    return best_individual, best_fitness

best_locations, best_fitness = genetic_algorithm(num_selected=8)

print("Best FQHC locations (indices):", best_locations + 1)
print("Best fitness (total residents served):", best_fitness)
print("Best FQHC locations (names):")
for index in best_locations:
    print(gdf.iloc[index]['Facility'])


Best FQHC locations (indices): [186  32 179 124 170 163 128  25]
Best fitness (total residents served): 506600
Best FQHC locations (names):
Access Family Care - Lamar (Medical)
Arthur Center Community Health
Compass Health - Butler Dental Clinic
Savannah Medical Clinic
Ozarks Community Health Center Dental
Jordan Valley Community Health - Mobile Medical Unit
Samuel U. Rodgers Health Center Downtown Campus
Affinia Healthcare (Administrative Office)


The approach I used is based on the idea of maximizing the total population served by selecting FQHC locations that cover the highest number of residents within their respective 30-mile buffer zones. This approach aligns with the goal of increasing healthcare access by focusing on areas with larger populations, thereby maximizing the impact of the FQHCs. The assumption here is that higher population densities indicate a greater need for healthcare services and that selecting locations with higher population counts will lead to better overall coverage.

**Rationale for Choosing This Approach:**  The approach based on population coverage assumes that the main objective is to serve as many people as possible, which is a practical and straightforward way to optimize FQHC placement. Locations with higher population densities have more residents that can benefit from the services offered by FQHCs, which is a critical factor for improving public health outcomes. The goal is to select locations that maximize the number of people who are likely to need healthcare services, and this method allows for prioritizing locations with the largest residential coverage.

**Assumptions in Computing Fitness:**
In computing the fitness of each location, I assumed that the population within a 30-mile radius of an FQHC was a good proxy for its need for healthcare services. I further assumed that all individuals within this buffer area would potentially access the FQHC if it were available. Therefore, the fitness of each location was directly related to the number of people within its service area. The fitness function computed the total number of residents served by an FQHC, and this value was maximized to identify the best locations.

**Fitness Function:**
The fitness function, in this case, was based on counting the number of people within the 30-mile buffer zone of each FQHC location. For each FQHC, I counted the number of residents that fell within the buffer using a spatial join operation. The metric I computed was the total population within the buffer zone, and this value was maximized because our goal was to serve the largest number of residents possible. By maximizing the fitness score (population served), I selected FQHC locations that could provide services to the most people, ensuring the most efficient allocation of resources for healthcare delivery.